In [9]:
! pip install mysql-connector-python
import mysql.connector
import pandas as pd
import getpass

distributed 1.21.8 requires msgpack, which is not installed.
astroid 2.3.3 has requirement six~=1.12, but you'll have six 1.11.0 which is incompatible.
You are using pip version 10.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [29]:
con = open('ref_con/db-prima', 'r').read().splitlines()
how = input('how to connect(local/public):')

mydb = mysql.connector.connect(
  host=con[0] if how == 'public' else con[1],
  user=con[2],
  password=con[3],
  database=con[4] 
)

mycursor = mydb.cursor(dictionary=True)

how to connect(local/public):local


In [11]:
def exec(q):
  mycursor.execute(q)
  return mycursor.fetchall()

def exc_q_transfer(Y,dp='183',T='M'):
    dpT = 'NULL'
    dpK = dp
    if(T == 'M'):
        dpT = dpK
        dpK = 'NULL'
    return exec('SELECT \
        inventori_transfer_logistik_detail.no_transfer,inventori_transfer_logistik.tgl_acc_permintaan AS tgl_transfer ,\
        masterf_katalog.nama_barang,qty_diberi as qty_diberi,hna_ppn_transfer hna_ppn,masterf_katalog.hna,\
        masterf_katalog.harga_terakhir,\
        SUM(hna_ppn_transfer * qty_diberi) AS total, md1.namaDepo AS penerima, md2.namaDepo AS pengirim,\
            jadbf2143.username,md1.id,md2.id, nama_kode_satuan_kecil,uraian, kode_obat, inventori_transfer_logistik.keterangan\
        FROM inventori_transfer_logistik\
        LEFT JOIN inventori_transfer_logistik_detail \
        ON inventori_transfer_logistik.no_transfer = inventori_transfer_logistik_detail.no_transfer\
        LEFT JOIN `masterf_depo` as md1 ON `md1`.`id` = `inventori_transfer_logistik`.`gudang_tujuan`\
        LEFT JOIN masterf_katalog on inventori_transfer_logistik_detail.kode_obat=masterf_katalog.kode\
        LEFT JOIN `masterf_depo` as md2 ON `md2`.`id` = `inventori_transfer_logistik`.`gudang_asal`\
        LEFT JOIN `user` as jadbf2143 ON `jadbf2143`.`id` = `inventori_transfer_logistik`.`acc_oleh`\
        INNER JOIN master_satuan_kecil e ON id_kode_satuan_kecil=kodeSatuanKecil\
        INNER JOIN master_sumber_referensi on (inventori_transfer_logistik.status=kode_ref and tipe_ref = "IST")\
        WHERE  1 = 1  AND inventori_transfer_logistik.status = 2  AND \
        DATE_FORMAT(inventori_transfer_logistik.tgl_acc_permintaan,"%Y-%m-%d %H:%i") \
        BETWEEN "'+Y+'-01-01 00:00" AND "'+Y+'-12-31 23:59" \
        AND kode_obat =IFNULL(NULL,kode_obat)\
        AND `inventori_transfer_logistik`.`gudang_tujuan` = IFNULL('+dpK+',`inventori_transfer_logistik`.`gudang_tujuan`)\
        AND `inventori_transfer_logistik`.`gudang_asal` = IFNULL('+dpT+',`inventori_transfer_logistik`.`gudang_asal`)  \
        GROUP BY inventori_transfer_logistik_detail.id ')

def do_in_out(Y,dp):
    ret = []
    ret.append(exc_q_transfer(Y,dp,'M'))
    ret.append(exc_q_transfer(Y,dp,'K'))
    return ret

def komparasiDenganDesc(D1,D2,r=0):
    return pd.merge(D1,D2,on=r,how='outer',indicator=True)

#bangun query
def bangun_q(DF,po,selisihM):
    trn = ''
    ret = ''
    for i,r in DF[['kode_obat','qty_diberi','no_transfer','keterangan']]\
[(DF['keterangan'].str.contains(pat=po,regex=True,case=False))].iterrows():
        ret += "update inventori_penerimaan_logistik_farmasi_detail set qty = qty - "+str(r['qty_diberi'])+" \
    where no_po = '"+po+"' and kode_obat = '"+r['kode_obat']+"';\n \
update log_stock_obatdepo set \
    jumlah_mutasi = jumlah_mutasi - "+str(r['qty_diberi'])+" where id_referensi = '"+po+"' \
    and kode_obat = '"+r['kode_obat']+"';\n"
        if(trn != r['no_transfer']):
            trn = r['no_transfer']
            ret += "UPDATE inventori_transfer_logistik\
 SET tgl_buat_permintaan = DATE_ADD(tgl_buat_permintaan, INTERVAL "+selisihM+" MONTH),\
tgl_acc_permintaan = DATE_ADD(tgl_acc_permintaan, INTERVAL "+selisihM+" MONTH) \
WHERE no_transfer = '"+trn+"';\n \
update log_stock_obatdepo set last_update = DATE_ADD(last_update, INTERVAL "+selisihM+" MONTH) \
where id_referensi = '"+trn+"';\n"
    return ret

In [30]:
R = do_in_out('2022','197')

In [31]:
DTM = pd.DataFrame.from_dict(R[0])
DTK = pd.DataFrame.from_dict(R[1])
DTMG = DTM[['kode_obat','keterangan','no_transfer','qty_diberi']].groupby(['kode_obat'],as_index=False).\
    agg({'qty_diberi':'sum','keterangan':'++'.join,'no_transfer':'++'.join})
DTKG = DTK[['kode_obat','keterangan','no_transfer','qty_diberi']].groupby(['kode_obat'],as_index=False).\
    agg({'qty_diberi':'sum','keterangan':'++'.join,'no_transfer':'++'.join})
DC = komparasiDenganDesc(DTMG,DTKG,'kode_obat')
DC.loc[:,'agg'] = DC['qty_diberi_x'].fillna(0) - DC['qty_diberi_y'].fillna(0)
DC[(DC['agg'] != 0)]

,kode_obat,qty_diberi_x,keterangan_x,no_transfer_x,qty_diberi_y,keterangan_y,no_transfer_y,_merge,agg


In [32]:
if DC[(DC['agg'] != 0)].empty:
    print(bangun_q(DTM,'00211/PO/PPO/04/2022','-1'))
else:
    print('not empty')

update inventori_penerimaan_logistik_farmasi_detail set qty = qty - 1.00     where no_po = '00211/PO/PPO/04/2022' and kode_obat = '020200381';
 update log_stock_obatdepo set     jumlah_mutasi = jumlah_mutasi - 1.00 where id_referensi = '00211/PO/PPO/04/2022'     and kode_obat = '020200381';
UPDATE inventori_transfer_logistik SET tgl_buat_permintaan = DATE_ADD(tgl_buat_permintaan, INTERVAL -1 MONTH),tgl_acc_permintaan = DATE_ADD(tgl_acc_permintaan, INTERVAL -1 MONTH) WHERE no_transfer = 'TRN/220512/208/197/00074';
 update log_stock_obatdepo set last_update = DATE_ADD(last_update, INTERVAL -1 MONTH) where id_referensi = 'TRN/220512/208/197/00074';
update inventori_penerimaan_logistik_farmasi_detail set qty = qty - 1.00     where no_po = '00211/PO/PPO/04/2022' and kode_obat = '020200399';
 update log_stock_obatdepo set     jumlah_mutasi = jumlah_mutasi - 1.00 where id_referensi = '00211/PO/PPO/04/2022'     and kode_obat = '020200399';



In [28]:
DC

,kode_obat,qty_diberi_x,keterangan_x,no_transfer_x,qty_diberi_y,keterangan_y,no_transfer_y,_merge,agg
0,01.021506217,10.00,H SPK 205 00871/PO/PPO/02/2022\n(160222),TRN/220311/208/197/00131,10.00,H SPK 205 00871/PO/PPO/02/2022\n(160222),TRN/220311/197/208/00129,both,0.00
1,01.023005518,2.00,H SPK 58 PO: 00748/PO/PPO/12/2021\nTUKAR ALAT ...,TRN/220106/208/197/00134,2.00,H SPK 58 NO.PO 00748/PO/PPO/12/2021\nTUKAR ALA...,TRN/220106/197/208/00133,both,0.00
2,01.023005618,1.00,H SPK 58 PO: 00748/PO/PPO/12/2021\nTUKAR ALAT ...,TRN/220106/208/197/00134,1.00,H SPK 58 NO.PO 00748/PO/PPO/12/2021\nTUKAR ALA...,TRN/220106/197/208/00133,both,0.00
3,0112360,2.00,00092/PO/PPO/04/2022,TRN/220512/208/197/00076,2.00,00092/PO/PPO/04/2022,TRN/220512/197/208/00075,both,0.00
4,01132450,1.00,h spk 331 02082/po/ppo/03/2022 os IBS tn.Addys...,TRN/220404/208/197/00071,1.00,h spk 331 02082/po/ppo/03/2022 os IBS Tn. addy...,TRN/220404/197/208/00070,both,0.00
5,0132990,1.00,h spk 331 02082/po/ppo/03/2022 os IBS tn.Addys...,TRN/220404/208/197/00071,1.00,h spk 331 02082/po/ppo/03/2022 os IBS Tn. addy...,TRN/220404/197/208/00070,both,0.00
6,020200014,1.00,h spk 331 02082/po/ppo/03/2022 os IBS tn.Addys...,TRN/220404/208/197/00071,1.00,h spk 331 02082/po/ppo/03/2022 os IBS Tn. addy...,TRN/220404/197/208/00070,both,0.00
7,020200381,1.00,h spk 332 os anyim cathlab 08/04/22,TRN/220512/208/197/00074,1.00,h spk 332 os anyim cathlab 8/4/22 00211/PO/PPO...,TRN/220512/197/208/00071,both,0.00
8,020200399,1.00,h spk 332 os anyim cathlab 08/04/22,TRN/220512/208/197/00074,1.00,h spk 332 os anyim cathlab 8/4/22 00211/PO/PPO...,TRN/220512/197/208/00071,both,0.00
9,020201276,1.00,h spk 331 02082/po/ppo/03/2022 os IBS tn.Addys...,TRN/220404/208/197/00071,1.00,h spk 331 02082/po/ppo/03/2022 os IBS Tn. addy...,TRN/220404/197/208/00070,both,0.00
